# ECE Chatbot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setting up packages and dependancies

In [ ]:
! pip install -qq -U langchain tiktoken pypdf faiss-gpu # packages for implementation
! pip install cohere openai
! pip install -qq -U transformers InstructorEmbedding sentence_transformers # for text embeddings
! pip install -qq -U accelerate bitsandbytes xformers einops # for optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 910.5 kB/s

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader # For loading pdf documents
from langchain.document_loaders import DirectoryLoader # for loading directories

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter # to help chunk the text

# prompts
from langchain import PromptTemplate, LLMChain # For prompts

# vector stores
from langchain.vectorstores import FAISS # FAISS embedding storage for retrival

# models
from langchain.llms import HuggingFacePipeline # Pipeline to import models
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA # For Q and A problems

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


LangChain: 0.0.350


In [ ]:
glob.glob('/content/pdfs')


['/content/pdfs']

## Setting up LLM

In [ ]:
class Config:
    # LLMs
    model_name = 'bloom' # bloom, falcon
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.15

    # splitting
    split_chunk_size = 500
    split_overlap = 10

    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'

    # similar passages
    k = 4

    # paths
    PDFs_path = '/content/pdfs'
    embeddings_path = '/content/faiss_index_hp'

In [ ]:
def model(model = Config.model_name):

    if model == 'bloom':
        model_repo = 'bigscience/bloom-7b1'

        tokenizer = AutoTokenizer.from_pretrained(model_repo)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
        )

        max_len = 1024

    return tokenizer, model, max_len

In [ ]:
%%time

tokenizer, model, max_len = model(model = Config.model_name)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 28.2 s, sys: 33.6 s, total: 1min 1s
Wall time: 3min 27s


In [ ]:
model.eval()

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear4bit(in_features=4096, out_features=12288, bias=True)
          (dense): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear4bit(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear4bit(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_a

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_length = max_len,
    temperature = Config.temperature,
    top_p = Config.top_p,
    repetition_penalty = Config.repetition_penalty
)

llm = HuggingFacePipeline(pipeline = pipe)

## Pre contextual learning question answering

### For making the answer more readable

In [ ]:
import re

def clean_chatbot_response(response, max_length=1000):
    """
    Cleans a chatbot response to make it more concise and focused, and returns it as a list of lines.

    :param response: The original response from the chatbot.
    :param max_length: Maximum length of the cleaned response.
    :return: A list of cleaned sentences from the response.
    """

    # Remove any script-like or log entries (e.g., "Loading PDF documents [ ] %%time")
    cleaned_response = re.sub(r'\[.*?\]|%%.*?$', '', response, flags=re.MULTILINE)

    # Break the response into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', cleaned_response)

    # Collect sentences up to the maximum length
    shortened_response = []
    current_length = 0
    for sentence in sentences:
        if current_length + len(sentence) <= max_length:
            shortened_response.append(sentence.strip())
            current_length += len(sentence)
        else:
            break

    return shortened_response

### What is relation extraction?

In [ ]:
%%time
query = "What is relation extraction ?"
answer = llm(query)
cleaned_response = clean_chatbot_response(answer)
for line in cleaned_response:
    print(line)

Relation Extraction (RE) aims to identify the semantic relations between two entities in a sentence.
The task can be divided into three subtasks: entity identification, relation classification and relation selection.
The first step of RE is named entity recognition (NER).
NER identifies all mentions of entities in a document.
In this paper we use Stanford CoreNLP toolkit for NER.
We used the following features for our system:
• Part-of-speech tags: This feature indicates whether an entity mention has been tagged as a noun phrase or not.
• Named Entity Recognition: This feature indicates whether an entity mention has been recognized by the NER module.
• Word embeddings: This feature represents the word embedding vector of each token in the input text.
It was obtained using GloVe model trained on Wikipedia data set.
• Character-level embedding: This feature represents the character-level embedding vectors of each token in the input text.
CPU times: user 1min 25s, sys: 117 ms, total: 1min

> It appears the bloom model has information on relation extraction therfore providing a good answer in terms of accuracy.

### Explain relation extraction in simple words

In [ ]:
query = "Explain relation extraction in simple words?"
answer = llm(query)
cleaned_response = clean_chatbot_response(answer)
for line in cleaned_response:
    print(line)

", "Relation Extraction: A Survey of the State-of-the-Art", Proceedings of the International Workshop on Relation Extraction (IWREx), pp.
1-8, 2008.
M.
Bouillon, S.
Drouin, and M.
Rastier.
"A survey of entity linking techniques for biomedical text mining", BMC Bioinformatics 10(1): p.
1, 2009.
J.
Breen, J.
G.
Hearst, and C.
Pustejovsky.
"The Semantic Web - a new approach to knowledge representation", Computer Science Reviews, vol.
43, no.
2, pages 3–32, June 2006.
D.
Cohen, E.
Lefevre, and F.
Sagot.
"Semantics-based information retrieval : an overview", Journal of Documentation, vol.
63, no.
4, pages 701–714, April 2004.
R.
Craven and T.
Kumlien.
"Ranking with vector space models", Information Processing Letters, vol.
62, no.
5, pages 31–34, 2001.
A.
Damon, B.
Katz, and L.
Schlerf.
"Multi-relational data mining : An introduction", ACM Computing Surveys, vol.
41, no.
6, pages 1–42, December 2005.
V.
Denisenko, V.
Grishman, and I.
Yihui.


> For this question however, the model is very bad at explaining the same topic in simpler terms, on;y providing random information

### What is perplexity?

In [ ]:
query = "Can you please explain what perplexity is ?"
answer = llm(query)
cleaned_response = clean_chatbot_response(answer)
for line in cleaned_response:
    print(line)

I am not sure if it has anything to do with the fact that my code is in a class, but I have tried several things and nothing seems to work.
Here are some of them:
public void setValue(String value) {
    this.value = value;
}

public String getValue() {
    return value;
}

public void setValue(int value) {
    this.value = Integer.toString(value);
}

public int getValue() {
    return Integer.parseInt(this.value);
}

I also tried this:
public void setValue(Object value) {
    this.value = (String)value;
}

public Object getValue() {
    return value;
}

But none seem to work.
Any help would be appreciated.
A:

You need to use getters/setters for your properties:
public void setValue(String value) {
    this.value = value;
}

public String getValue() {
    return value;
}

public void setValue(int value) {
    this.value = Integer.toString(value);
}

public int getValue() {
    return Integer.parseInt(this.value);
}


> The boolm LLM has just spit out unchorent chunks of code, which has nothing to do with perplexity, in our case the relation to the complexity of the model

### Please explain Perplexity in simple words

In [ ]:
query = "Please explain perplexity in simple words ?"
answer = llm(query)
cleaned_response = clean_chatbot_response(answer)
for line in cleaned_response:
    print(line)

I am not able to understand the concept of perplexity.
Can anyone please help me with this?
Thanks

A:

The term is used when you are trying to measure how well a model can predict new data, and it has nothing to do with your understanding of probability.
For example, if I have two models:
Model 1: A = ; B = ; C = ; D = 
Model 2: A = ; B = ; C = ; D = 
Then Model 1 will be better at predicting the next value than Model 2 because it predicts more values that were seen before (A,B,C) while Model 2 only predicted one (D).
This is why we use cross-validation for training our models so they don't overfit on their own data: We split up the dataset into several parts, train them separately using different subsets, then test each part against all other parts.
This way we get an unbiased estimate of how good our model really is.
A:

Perplexity is a measure of how much information there is about the distribution of the data given by the model.


> The simplified version is moer coherent than the unsimplified one

## Loading PDF documents

In [ ]:
loader = DirectoryLoader(
    Config.PDFs_path,
    glob="./*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|██████████| 2/2 [00:41<00:00, 20.75s/it]

CPU times: user 38.3 s, sys: 237 ms, total: 38.5 s
Wall time: 41.5 s


## Splitting and chunking documents

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = Config.split_chunk_size,
    chunk_overlap = Config.split_overlap
)

texts = splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 6925 chunks from 1140 pages


In [ ]:
# download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
             model_name = Config.embeddings_model_repo,
             model_kwargs = {"device": "cuda"}
  )
vectordb = FAISS.from_documents(documents = texts, embedding = embeddings)# Creating a vector data base using FAISS


vectordb.save_local("faiss_index_hp")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# File name
file_name = "faiss_index_hp"

# Get the current working directory
current_directory = os.getcwd()

# Full path to the file
file_path = os.path.join(current_directory, file_name)

print(f"The full path to the file is: {file_path}")

The full path to the file is: /content/faiss_index_hp


In [ ]:
# download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = Config.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

### load vector DB embeddings
ece = FAISS.load_local(
    Config.embeddings_path,
    embeddings
)

load INSTRUCTOR_Transformer
max_seq_length  512


## Checking if the embeddings loaded correctly

In [ ]:
ece.similarity_search_with_score("perplexity")

[(Document(page_content='the test set. Any kind of knowledge of the test set can cause the perplexity to be\nartiﬁcially low. The perplexity of two language models is only comparable if they\nuse identical vocabularies.\nAn (intrinsic) improvement in perplexity does not guarantee an (extrinsic) im-\nprovement in the performance of a language processing task like speech recognition', metadata={'source': '/content/pdfs/ed3book.pdf', 'page': 46}),
  0.9348018),
 (Document(page_content='Xu et al. (2019) also give a useful baseline algorithm that itself has quite high\nperformance in measuring perplexity: train an RNN language model on the data,\nand compute the log likelihood of sentence siin two ways, once given the preceding\ncontext (conditional log likelihood) and once with no context (marginal log likeli-\nhood). The difference between these values tells us how much the preceding context\nimproved the predictability of si, a predictability measure of coherence.', metadata={'source': '

In [ ]:
ece.max_marginal_relevance_search("perplexity")

[Document(page_content='the test set. Any kind of knowledge of the test set can cause the perplexity to be\nartiﬁcially low. The perplexity of two language models is only comparable if they\nuse identical vocabularies.\nAn (intrinsic) improvement in perplexity does not guarantee an (extrinsic) im-\nprovement in the performance of a language processing task like speech recognition', metadata={'source': '/content/pdfs/ed3book.pdf', 'page': 46}),
 Document(page_content='(SPARQL queries) for those questions answerable using Freebase. C OMPLEX WEB-\nQUESTIONS augments the dataset with compositional and other kinds of complex\nquestions, resulting in 34,689 questions, along with answers, web snippets, and\nSPARQL queries (Talmor and Berant, 2018).\nLet’s assume we’ve already done the stage of entity linking introduced in the\nprior section. Thus we’ve mapped already from a textual mention like Ada Lovelace', metadata={'source': '/content/pdfs/ed3book.pdf', 'page': 294}),
 Document(page_conte

In [ ]:
ece.similarity_search("perpexity")

[Document(page_content='40 CHAPTER 3 • N- GRAM LANGUAGE MODELS\nor machine translation. Nonetheless, because perplexity often correlates with such\nimprovements, it is commonly used as a quick check on an algorithm. But a model’s\nimprovement in perplexity should always be conﬁrmed by an end-to-end evaluation\nof a real task before concluding the evaluation of the model.\n3.3 Sampling sentences from a language model\nOne important way to visualize what kind of knowledge a language model embodies', metadata={'source': '/content/pdfs/ed3book.pdf', 'page': 47}),
 Document(page_content='Xu et al. (2019) also give a useful baseline algorithm that itself has quite high\nperformance in measuring perplexity: train an RNN language model on the data,\nand compute the log likelihood of sentence siin two ways, once given the preceding\ncontext (conditional log likelihood) and once with no context (marginal log likeli-\nhood). The difference between these values tells us how much the preceding cont

> All the searces and their similarity contains some sort of information from about perplexity.

## Prompting

In [ ]:

prompt_template = """
Please provide a detailed and accurant answer.
If there is not enough infomration to answer say "I am sorry I do not know".
Keep the defenitions and explanation in scientific domain try to give examples if possible.
Use context to answer the question at the end.

Context: {context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)


## Retrival

In [ ]:
retriever = ece.as_retriever(search_kwargs = {"k": Config.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

### Text processing for better presentation

In [ ]:
import textwrap

def text_wrap(text, width=125):
    # Improved wrapping to handle long words and preserve existing formatting
    wrapped_lines = []
    for line in text.split('\n'):
        if len(line) > width:
            wrapped_lines.extend(textwrap.wrap(line, width=width))
        else:
            wrapped_lines.append(line)
    return '\n'.join(wrapped_lines)

def process_llm_response(llm_response):
    # Wrap the response text for better readability
    ans = text_wrap(llm_response['result'])

    # Aggregate source references
    if llm_response['source_documents']:
        sources = {}
        for source in llm_response['source_documents']:
            source_name = source.metadata['source'].split('/')[-1][:-4]
            page = str(source.metadata['page'])
            if source_name in sources:
                sources[source_name].add(page)
            else:
                sources[source_name] = {page}

        formatted_sources = ' \n'.join(
            f"{source} - pages: {', '.join(sorted(sources[source]))}"
            for source in sources
        )
        source_section = f'\n\nSources:\n{formatted_sources}'
    else:
        source_section = '\n\nSources: None found'

    return f"{ans}{source_section}"


def ans(query):
    llm_response = qa_chain(query)
    answer = process_llm_response(llm_response)

    return answer


## Evaluation

### What is relation extraction

In [ ]:
question = "What is relation extraction?"
print(ans(question))

Relation extraction is the process of extracting
facts about relationships between entities from
text. For instance, given two sentences:

A: John was born in New York City.
B: John's father was born in New York City.

the task would be to determine whether the second
sentence describes a relationship between the
first two entities, such as "John's father is
John's father". If it does, then this fact should
be extracted into a new relation named
"father_of".

Sources:
ed3book - pages: 436, 440, 446, 447


> time : 9s

> Information :

### Explain relation extraction in simple words

In [ ]:
question = "Explain relation extraction in simple words"
print(ans(question))

Relation extraction is the process of extracting
facts about relationships between two entities
from text. It has been widely studied as a sub-
task of natural language processing. There are
three main types of relation extraction:
supervised, semi-supervised, and unsupervised. In
supervised relation extraction, a large amount of
manually annotated training data is needed. Semi-
and unsupervised relation extraction does not need
manual annotation but requires some form of
supervision.

Sources:
ed3book - pages: 440, 446, 447 
NLTK - pages: 307


> Time: 9 s

> Information:

### What is Perplexity?

In [ ]:
question = "What is perplexity "
print(ans(question))

Perplexity measures the amount by which the probability distribution over the
words in a text deviates from uniform. In other words, it quantifies the extent to
which the words in a text tend to cluster together or spread out across the text.
Intuitively, the more clustered the words are, the less likely it is that you’ll
find them in random order. This makes sense because when you’re reading a book, you
expect to encounter certain words often, but you don’t expect to see those same words
randomly scattered throughout the text. If you were to read a book where every single
word was equally likely to appear anywhere within the text, then your chances of
encountering a particular word would be exactly 50% regardless of its position in the
text. However, since most books have some sort of theme or structure, it’s unlikely
that you’d encounter a word just randomly throughout the text. Instead, you’d expect
to find words related to the topic of the book in predictable patterns. For example,

> Time : 53s

> Information: The information is very specific and factual.

### Please explain Perplexity in simple words

In [ ]:
question = "Please explain Perplexity in simple words"
print(ans(question))

Perplexity is a measure of how well a model predicts the next word given the previous one. It is calculated by taking the log
likelihood ratio between the observed sequence and the predicted sequence. For example, consider the following sentence:

The man who was born yesterday died today.

This sentence has three words, namely, man, who, born, yesterday, died, today. If you take the logarithm of the probability
of each word given its preceding word, then you get the following:

Log(P(man|yesterday)) = −0.5 + 1.5 log(1/0.25) = −1.5
Log(P(who|born)) = −2.5 + 2.5 log(1/6) = −3.5
Log(P(yesterday|died)) = −4.5 + 3.5 log(1/1.75) = −3.5
Log(P(today|died)) = 5.5 + 4.5 log(1/2.25) = 4.5

Now, suppose you have another model with the same vocabulary as above but where the probabilities of the words are different.
Then, the log-likelihood ratios would be:

Log(P(man|yesterday)) = −0.5 + 1.5 log(2/1.25) = −1.5
Log(P(who|born)) = −2.5 + 2.5 log(2/3.5) = −3.5
Log(P(yesterday|died)) = −4.5 + 3.5 log(

> Time : 37s

> Information:  The text genrated is mostly facutal however the numbers generate are incorrect.

### Handling out of document questions

In [ ]:
question = "Give me R code for understanding decision tree"
print(ans(question))

#!/usr/bin/env python3
import sys
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.clustering import DBSCAN
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.externals import joblib
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import add_sigmoi

> Time : 53s

> Information: Is unfactual and irrelevant

In [ ]:
question = "What is the best dinner place in detroit"
print(ans(question))

1) The best Italian food is located near a museum.
2) It costs $25 for two people.
3) It’s reasonably priced.
4) It’s also good value for money.
5) It’s an excellent choice for a romantic evening.
6) It’s a great spot for a date night.
7) It’s a nice place to go out on a Saturday night.
8) It’s a good place to eat when you want to get away from it all.
9) It’s a good place to have a quiet meal after work.
10) It’s a good place to take your kids.
11) It’s a good place to meet new friends.
12) It’s a good place to hang out with your family.
13) It’s a good place to spend time with your partner.
14) It’s a good place to enjoy some quality time together.
15) It’s a good place to relax and unwind.
16) It’s a good place to celebrate special occasions.
17) It’s a good place to treat yourself.
18) It’s a good place to dine in style.
19) It’s a good place to enjoy fine dining.
20) It’s a good place to enjoy a delicious meal.
21) It’s a good place to enjoy a hearty meal.
22) It’s a good place to

> Time: 33s

> Information: Not factual, but relatively specific to the question asked